In [ ]:
import sys
sys.path.append("../../")
import json
import numpy as np
from PIL import Image
import pandas as pd
from DMIN.utils import bytes_to_image, get_config
from datasets import load_dataset
import hnswlib

In [ ]:
result_df = pd.read_parquet("./output_dir/test_dataset.parquet")
train_df = load_dataset("huaweilin/DMin_mixed_datasets_8846", split="train").to_pandas()

In [ ]:
config = get_config("config.json")
p = hnswlib.Index(space='ip', dim=config.compression.K*config.data.num_estimate_steps)
index_path = f"./{config.influence.cache_path}/index_K{config.compression.K}.bin"
p.load_index(index_path)
p.set_ef(500)

In [ ]:
def concat_images(base_image, img_list):
    widths_0, heights = base_image.size
    widths_1, heights = zip(*(i.size for i in img_list))

    gap = 5
    large_gap = 120

    total_width = widths_0 + sum(widths_1) + gap*(len(img_list) - 1) + large_gap
    max_height = max(heights)
    new_img = Image.new('RGB', (total_width, max_height), color = (255,255,255))
    
    x_offset = 0
    new_img.paste(base_image, (x_offset, 0))
    x_offset += base_image.width + large_gap

    for img in img_list:
        new_img.paste(img, (x_offset, 0))
        x_offset += img.width + gap

    return new_img


for idx, data_dict in result_df.iterrows():
    print("-"*20, idx, "-"*20)
    test_loss_grad = np.hstack(data_dict["loss_grad_compressed_list"][0].tolist())
    indices = p.knn_query(test_loss_grad, k=20)[0][0]
    print(indices)
    base_image = bytes_to_image(result_df.iloc[idx]["image"])
    concat_images(base_image.resize((512, 512)), [bytes_to_image(train_df.iloc[x]["image"]["bytes"]).resize((512, 512)) for x in indices]).show()